In [ ]:
# TODO correlation between omikron_pg and omikron

In [ ]:
import argparse
import networkx as nx
import numpy as np
import scipy as sp
import tensorflow as tf
import tensorflow.contrib.eager as eager
import my
import objectives
import gnn_lg

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.enable_eager_execution(config)

In [ ]:
args = argparse.Namespace()
args.activation = tf.keras.activations.relu
args.depth = 10
args.device = '/cpu:0'
# args.device = '/device:GPU:0'
# args.graph = 'soc-Epinions1'
# args.graph = 'soc-Slashdot0811'
# args.graph = 'soc-Slashdot0902'
args.graph = 'barabasi-albert-100-3'
args.lambda_node = 0.0
args.lambda_edge = 0.0
args.lambda_entropy = 0.0
args.n_features = 16
args.n_iterations = 500
args.n_machines = 10
args.n_samples = 5
args.radius = 3

# parser = argparse.ArgumentParser()
# parser.add_argument('--depth', type=int, default=10)
# parser.add_argument('--device', type=str, default='/cpu:0')
# parser.add_argument('--graph', type=str, default='soc-Epinions1')
# parser.add_argument('--lambda_node', type=float, default=1)
# parser.add_argument('--lambda_edge', type=float, default=1)
# parser.add_argument('--lambda_entropy', type=float, default=1)
# parser.add_argument('--n-features', type=int, default=16)
# parser.add_argument('--n-iterations', type=int, default=500)
# parser.add_argument('--n-machines', type=int, default=10)
# parser.add_argument('--radius', type=int, default=3)
# args = parser.parse_args()

keys = sorted(vars(args).keys())
run_id = 'ipynb-' + '-'.join('%s-%s' % (key, str(getattr(args, key))) for key in keys if key != 'device')
writer = tf.contrib.summary.create_file_writer('runs/' + run_id)
writer.set_as_default()

In [ ]:
if args.graph.startswith('barabasi-albert'):
    n, m = tuple(map(int, args.graph.split('-')[-2:]))
    g = nx.barabasi_albert_graph(n, m)
else:
    g = my.read_edgelist('data/' + args.graph, fill=True)

In [ ]:
import imp
gnn_lg = imp.reload(gnn_lg)
objectives = imp.reload(objectives)

In [ ]:
with tf.device(args.device):
    objective = objectives.Objective(g, args.lambda_node, args.lambda_edge, args.lambda_entropy, args.n_machines, args.n_samples)
    network = gnn_lg.GNN(g, (1,) + (args.n_features,) * args.depth, args.n_machines, args.radius, args.activation, args.device)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-1)

In [ ]:
global_step = tf.train.get_or_create_global_step()
for i in range(args.n_iterations):
    global_step.assign_add(1)
    with tf.contrib.summary.record_summaries_every_n_global_steps(1):
        with eager.GradientTape() as tape:
            z = network(None, training=True)
            omikron_pg = objective(z, training=True)
        gradients = tape.gradient(omikron_pg, network.variables)
        optimizer.apply_gradients(zip(gradients, network.variables), global_step=tf.train.get_or_create_global_step())

        tf.contrib.summary.scalar('objective-pg', omikron)

        omikron, r, b_node, b_edge, entropy = objective(z)
        tf.contrib.summary.scalar('objective', omikron)
        tf.contrib.summary.scalar('replication-factor', r)
        tf.contrib.summary.scalar('node-balancedness', b_node)
        tf.contrib.summary.scalar('edge-balancedness', b_edge)
        tf.contrib.summary.scalar('entropy', entropy)

        if (i + 1) % 10 == 0:
            print('[iteration %d]omikron_pg %0.3f | omikron %0.3f | r %0.3f | b_node %0.3f | b_edge %0.3f | entropy %0.3f' %
                  (i + 1, omikron_pg, omikron, r, b_node, b_edge, entropy))

In [ ]:
# ckpt = eager.Checkpoint(model=network)
# ckpt.save('./models/network')